# Minimal Academic Notebook: Dissipative Evolution (2×2)

This notebook demonstrates **dissipative (contractive semigroup) evolution** in a minimal 2×2 TNFR system.
We compute the key structural metrics:

- **νf** (structural frequency, Hz_str): reorganization rate from the Lindblad ΔNFR generator
- **C_min**: minimal coherence eigenvalue (structural stability threshold)
- **d_coh**: coherence dissimilarity between evolved density operators

The dissipative evolution follows the **Lindblad master equation** in TNFR form:

$$\frac{\partial \rho}{\partial t} = \nu_f \cdot \mathcal{L}[\rho]$$

where $\mathcal{L}$ is the Lindblad superoperator (ΔNFR generator in Liouville space).

**Reproducibility**: Fixed random seed for deterministic results.

In [ ]:
"""Import public TNFR mathematics API and NumPy for reproducibility."""
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Public TNFR mathematics API
from tnfr.mathematics import (
    HilbertSpace,
    CoherenceOperator,
    ContractiveDynamicsEngine,
    build_lindblad_delta_nfr,
)
from tnfr.mathematics.metrics import dcoh

# Reproducibility seed
SEED = 42
rng = np.random.default_rng(SEED)

print(f"TNFR Dissipative Evolution - 2×2 System (seed={SEED})")

## 1. System Setup: 2×2 Hilbert Space

Initialize a minimal 2-dimensional Hilbert space for dissipative TNFR dynamics.

In [ ]:
# Initialize 2×2 Hilbert space
dim = 2
hilbert_space = HilbertSpace(dimension=dim)

print(f"Hilbert Space dimension: {hilbert_space.dimension}")
print(f"Canonical basis shape: {hilbert_space.basis.shape}")
print(f"Liouville space dimension: {dim**2} × {dim**2}")

## 2. Build Lindblad ΔNFR Generator

Create a **Lindblad generator** (dissipative ΔNFR) with:
- **Hamiltonian**: Hermitian coherent part
- **Collapse operators**: Drive dissipation and decoherence

In [ ]:
# Structural frequency
nu_f = 1.0  # Hz_str

# Build Hamiltonian (Hermitian)
H = np.array([
    [1.0, 0.2],
    [0.2, -0.5]
], dtype=complex)

# Verify Hermiticity
hermitian_check = np.allclose(H, H.conj().T)
print(f"Hamiltonian (Hermitian check: {hermitian_check}):")
print(H)

# Build collapse operators (drive dissipation)
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

# Scale collapse operators
gamma_x = 0.1  # Dissipation rate
gamma_z = 0.05

L1 = np.sqrt(gamma_x) * sigma_x
L2 = np.sqrt(gamma_z) * sigma_z

print(f"\nCollapse operators:")
print(f"L₁ (σₓ-like, γ={gamma_x}):")
print(L1)
print(f"\nL₂ (σz-like, γ={gamma_z}):")
print(L2)

In [ ]:
# Build Lindblad ΔNFR generator (superoperator in Liouville space)
lindblad_generator = build_lindblad_delta_nfr(
    hamiltonian=H,
    collapse_operators=[L1, L2],
    nu_f=nu_f,
    scale=1.0,
    ensure_trace_preserving=True,
    ensure_contractive=True,
)

print(f"\nLindblad ΔNFR Generator shape: {lindblad_generator.shape}")
print(f"Acts on vectorized density operators: {dim**2} × {dim**2}")
print(f"\nνf (structural frequency): {nu_f} Hz_str")

# Check trace preservation (sum of each row should be 0 for trace-preserving)
row_sums = lindblad_generator.sum(axis=1)
trace_preserving = np.allclose(row_sums, 0.0, atol=1e-9)
print(f"Trace-preserving check: {trace_preserving}")

## 3. Coherence Operator and C_min

Build a coherence operator and extract **C_min** (minimal eigenvalue).

In [ ]:
# Create coherence operator with positive spectrum
c_spectrum = np.array([0.3, 0.7])  # Positive eigenvalues
coherence_op = CoherenceOperator(c_spectrum)

print("Coherence Operator:")
print(coherence_op.matrix)
print(f"\nEigenvalues: {coherence_op.eigenvalues.real}")

# Extract C_min (minimal coherence threshold)
c_min = coherence_op.c_min
print(f"\nC_min (minimal coherence): {c_min:.4f}")
print(f"Structural stability threshold: {c_min:.4f}")

## 4. Contractive Dynamics Engine

Initialize the dissipative evolution engine with the Lindblad generator.

In [ ]:
# Create contractive dynamics engine
engine = ContractiveDynamicsEngine(
    generator=lindblad_generator,
    hilbert_space=hilbert_space,
    ensure_contractive=True,
    use_scipy=True,  # Use SciPy for matrix exponential
)

print(f"Dynamics Engine: {type(engine).__name__}")
print(f"Generator shape: {engine.generator.shape}")
print(f"Hilbert space dimension: {engine.hilbert_space.dimension}")
print(f"Contractive: verified during initialization")

## 5. Evolve Density Operators

Evolve an initial density operator through the dissipative dynamics.

In [ ]:
# Initial density operator: pure state |ψ⟩⟨ψ| with ψ = (1, 1)/√2
psi0 = np.array([1.0, 1.0], dtype=complex)
psi0 = psi0 / np.linalg.norm(psi0)
rho0 = np.outer(psi0, psi0.conj())

print("Initial density operator ρ₀:")
print(rho0)
print(f"\nTrace(ρ₀): {np.trace(rho0):.6f}")
print(f"Purity: {np.trace(rho0 @ rho0).real:.6f}")

# Evolve through multiple time steps
dt = 0.1
steps = 20
trajectory = engine.evolve(rho0, steps=steps, dt=dt)

print(f"\nEvolved trajectory shape: {trajectory.shape}")
print(f"Total evolution time: {steps * dt:.2f}")

## 6. Extract States and Compute Metrics

Extract density operators at different times and compute traces and purities.

In [ ]:
# Extract density operators at different times
rho_t0 = trajectory[0]
rho_t10 = trajectory[10]
rho_t20 = trajectory[20]

# Compute traces (should be preserved)
trace_t0 = np.trace(rho_t0)
trace_t10 = np.trace(rho_t10)
trace_t20 = np.trace(rho_t20)

# Compute purities (should decrease due to dissipation)
purity_t0 = np.trace(rho_t0 @ rho_t0).real
purity_t10 = np.trace(rho_t10 @ rho_t10).real
purity_t20 = np.trace(rho_t20 @ rho_t20).real

print("Density operator properties:")
print(f"\nt=0.0:")
print(f"  Trace: {trace_t0:.6f}")
print(f"  Purity: {purity_t0:.6f}")

print(f"\nt=1.0:")
print(f"  Trace: {trace_t10:.6f}")
print(f"  Purity: {purity_t10:.6f}")

print(f"\nt=2.0:")
print(f"  Trace: {trace_t20:.6f}")
print(f"  Purity: {purity_t20:.6f}")

# Trace preservation check
print(f"\nTrace preservation: all traces ≈ 1.0")

## 7. Compute Coherence Dissimilarity (d_coh)

To compute d_coh for density operators, we extract dominant eigenvectors (pure state approximation).

In [ ]:
# Helper: extract dominant eigenvector from density operator
def dominant_eigenvector(rho):
    """Extract dominant eigenvector from density operator."""
    eigvals, eigvecs = np.linalg.eigh(rho)
    idx = np.argmax(eigvals.real)
    return eigvecs[:, idx]

# Extract dominant eigenvectors (approximate pure states)
psi_from_t0 = dominant_eigenvector(rho_t0)
psi_from_t10 = dominant_eigenvector(rho_t10)
psi_from_t20 = dominant_eigenvector(rho_t20)

# Compute d_coh between states
d_coh_0_10 = dcoh(psi_from_t0, psi_from_t10, coherence_op, normalise=True)
d_coh_0_20 = dcoh(psi_from_t0, psi_from_t20, coherence_op, normalise=True)
d_coh_10_20 = dcoh(psi_from_t10, psi_from_t20, coherence_op, normalise=True)

print("Coherence Dissimilarity (d_coh):")
print(f"  d_coh(ρ₀, ρ_{1.0}): {d_coh_0_10:.6f}")
print(f"  d_coh(ρ₀, ρ_{2.0}): {d_coh_0_20:.6f}")
print(f"  d_coh(ρ_{1.0}, ρ_{2.0}): {d_coh_10_20:.6f}")

# Compute d_coh trajectory
dcoh_trajectory = []
for i in range(len(trajectory)):
    psi_i = dominant_eigenvector(trajectory[i])
    d = dcoh(psi_from_t0, psi_i, coherence_op, normalise=True)
    dcoh_trajectory.append(d)

dcoh_trajectory = np.array(dcoh_trajectory)
print(f"\nd_coh trajectory computed for {len(trajectory)} time steps")

## 8. Compute Purity Trajectory

Track purity loss due to dissipation.

In [ ]:
# Compute purity for all time steps
purity_trajectory = []
trace_trajectory = []

for i in range(len(trajectory)):
    rho_i = trajectory[i]
    purity = np.trace(rho_i @ rho_i).real
    trace = np.trace(rho_i).real
    purity_trajectory.append(purity)
    trace_trajectory.append(trace)

purity_trajectory = np.array(purity_trajectory)
trace_trajectory = np.array(trace_trajectory)

print(f"Purity decay:")
print(f"  Initial: {purity_trajectory[0]:.6f}")
print(f"  Final: {purity_trajectory[-1]:.6f}")
print(f"  Loss: {purity_trajectory[0] - purity_trajectory[-1]:.6f}")
print(f"\nTrace preservation:")
print(f"  Mean trace: {trace_trajectory.mean():.6f}")
print(f"  Std dev: {trace_trajectory.std():.9f}")

## 9. Visualization

Generate figures showing:
- Purity decay (signature of dissipation)
- Coherence dissimilarity d_coh(t)

In [ ]:
# Time array
time = np.linspace(0, steps * dt, steps + 1)

# Figure: Dissipative evolution metrics
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Plot purity decay
axes[0].plot(time, purity_trajectory, 'b-', linewidth=2.5, label='Purity Tr(ρ²)')
axes[0].axhline(y=1.0, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label='Pure state')
axes[0].axhline(y=0.5, color='red', linestyle=':', linewidth=1.5, alpha=0.7, label='Maximally mixed (2D)')
axes[0].set_xlabel('Time (structural units)', fontsize=12)
axes[0].set_ylabel('Purity', fontsize=12)
axes[0].set_title('Dissipative Evolution: Purity Decay', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.4, 1.05])

# Plot d_coh trajectory
axes[1].plot(time, dcoh_trajectory, 'g-', linewidth=2.5)
axes[1].axhline(y=c_min, color='orange', linestyle=':', linewidth=2, label=f'C_min = {c_min:.3f}')
axes[1].set_xlabel('Time (structural units)', fontsize=12)
axes[1].set_ylabel('d_coh (dissimilarity)', fontsize=12)
axes[1].set_title('Coherence Dissimilarity Evolution', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

# Save figure
output_dir = Path('.')
output_path = output_dir / '02_dissipative_evolution.png'
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\nFigure saved: {output_path}")

plt.show()

## 10. Summary of Structural Metrics

Key TNFR quantities computed in this notebook:

In [ ]:
print("=" * 60)
print("TNFR DISSIPATIVE EVOLUTION - STRUCTURAL METRICS SUMMARY")
print("=" * 60)
print(f"\nSystem: 2×2 Hilbert space (Lindblad ΔNFR generator)")
print(f"Dissipation: 2 collapse operators (σₓ, σz)")
print(f"\nStructural Frequency (νf): {nu_f:.4f} Hz_str")
print(f"Minimal Coherence (C_min): {c_min:.4f}")
print(f"\nPurity Evolution:")
print(f"  Initial: {purity_trajectory[0]:.6f}")
print(f"  Final:   {purity_trajectory[-1]:.6f}")
print(f"  Loss:    {purity_trajectory[0] - purity_trajectory[-1]:.6f}")
print(f"\nCoherence Dissimilarity (d_coh):")
print(f"  Initial → t=1.0: {d_coh_0_10:.6f}")
print(f"  Initial → t=2.0: {d_coh_0_20:.6f}")
print(f"  Maximum d_coh:   {dcoh_trajectory.max():.6f}")
print(f"\nEvolution: {steps} steps × dt={dt} = {steps*dt:.2f} structural time units")
print(f"Trace preservation: {trace_trajectory.mean():.6f} (verified)")
print(f"Contractive semigroup: purity monotonically decreases")
print("=" * 60)

## Notes

- **Dissipative evolution**: Generated by Lindblad ΔNFR superoperator $\mathcal{L}$
- **Contractive semigroup**: Purity monotonically decreases, trace is preserved
- **νf (structural frequency)**: Sets the reorganization rate for dissipative dynamics
- **C_min**: Structural stability threshold
- **d_coh**: Measures coherence divergence during dissipation
- **Collapse operators**: Drive decoherence and relaxation
- All operations use **public TNFR API** from `tnfr.mathematics` module